# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

In [5]:
df = pd.read_csv("event_datafile_new.csv")

In [6]:
df.dtypes

artist            object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
sessionId          int64
song              object
userId             int64
dtype: object

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)
except Exception as e:
    print(e)

#### Set Keyspace

In [9]:
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

# Create queries to ask the following three questions of the data

## 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create Table

In [10]:
create_app_hist_query = "CREATE TABLE IF NOT EXISTS app_history"
create_app_hist_query = create_app_hist_query + '''(sessionId int, 
                    itemInSession int, 
                    artist text, 
                    length float, 
                    song text, 
                    PRIMARY KEY(sessionId, itemInSession))'''

# execute query
try:
    session.execute(create_app_hist_query)
except Exception as e:
    print(e)


#### Test that the table is created

In [11]:
test = "SELECT COUNT(*) from app_history ;"
try:
    t1 = session.execute(test)
except Exception as e:
    print(e)
    
for row in t1:
    print (row)

Row(count=0)


#### Insert into table

In [12]:
# Indices and type
col_names = ['sessionId', 'itemInSession', 'artist', 'length', 'song']
app_hist_indices = [df.columns.get_loc(col) for col in col_names]
app_hist_types = [int, int, str, float, str]

In [13]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line[8])
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO app_history(sessionId, itemInSession, artist, length, song)"
        query = query + "VALUES(%s, %s, %s, %s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, tuple(type(line[x]) for x, type in zip(app_hist_indices, app_hist_types)))

#### Select to verify that the data have been inserted into each table

In [14]:
# TO DO : Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
select_query1 = """
                SELECT artist, song, length 
                FROM app_history 
                WHERE sessionId = 338 AND itemInSession = 4
                """

# execute
try:
    rows = session.execute(select_query1)
except Exception as e:
    print(e)
    
# print
for row in rows:
    print(row)
    print("*" * 100)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)
****************************************************************************************************


# COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

## 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    



#### Create Table

In [15]:
create_user_query = "CREATE TABLE IF NOT EXISTS user_session_history"
create_user_query = create_user_query + '''(userId int,
                                            sessionId int,
                                            itemInSession int,
                                            firstName text,
                                            lastName text,
                                            artist text,
                                            song text,
                                            PRIMARY KEY((userId, sessionId),itemInSession)
                                            )'''

# execute query
try:
    session.execute(create_user_query)
except Exception as e:
    print(e)


#### Test that the table is created

In [16]:
test2 = "SELECT COUNT(*) from user_session_history ;"
try:
    t2 = session.execute(test2)
except Exception as e:
    print(e)
    
for row in t2:
    print (row)

Row(count=0)


#### Insert into table

In [17]:
# Indices and type
col_names2 = ['userId', 'sessionId', 'itemInSession', 'firstName', 'lastName', 'artist', 'song']
user_hist_indices = [df.columns.get_loc(col) for col in col_names2]
user_hist_types = [int, int, int, str, str, str, str]

In [18]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line[8])
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO user_session_history(userId, sessionId, itemInSession, firstName, lastName, artist, song)"
        query = query + "VALUES(%s, %s, %s, %s,%s, %s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, tuple(type(line[x]) for x, type in zip(user_hist_indices, user_hist_types)))

In [19]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

query2 = """SELECT artist, song, firstName, lastName
           FROM user_session_history
           WHERE userId = 10 AND sessionId = 182
           ORDER BY itemInSession ASC
           """


# execute
try:
    rows = session.execute(query2)
except Exception as e:
    print(e)
    
# print
for row in rows:
    print(row)
    print("*" * 100)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
****************************************************************************************************
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
****************************************************************************************************
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
****************************************************************************************************
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')
****************************************************************************************************


## 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create Table

In [20]:
create_song_hist_query = "CREATE TABLE IF NOT EXISTS song_history"
create_song_hist_query = create_song_hist_query + '''(song text,
                                                    userId int,
                                                    firstName text,
                                                    lastName text,
                                                    PRIMARY KEY(song, userId))'''

# execute query
try:
    session.execute(create_song_hist_query)
except Exception as e:
    print(e)


#### Test that the table is created

In [21]:
test3 = "SELECT COUNT(*) from song_history ;"
try:
    t3 = session.execute(test3)
except Exception as e:
    print(e)
    
for row in t3:
    print (row)

Row(count=0)


#### Insert into table

In [22]:
# Indices and type
col_names3 = ['song', 'userId', 'firstName', 'lastName']
song_hist_indices = [df.columns.get_loc(col) for col in col_names3]
song_hist_types = [str, int, str, str]

In [23]:
df.head(1)

,artist,firstName,gender,itemInSession,lastName,length,level,location,sessionId,song,userId
0,Stephen Lynch,Jayden,M,0,Bell,182.85669,free,"Dallas-Fort Worth-Arlington, TX",829,Jim Henson's Dead,91


In [24]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #print(line[8])
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_history(song, userId, firstName, lastName)"
        query = query + "VALUES(%s, %s, %s, %s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, tuple(type(line[x]) for x, type in zip(song_hist_indices, song_hist_types)))

In [25]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

query3 = """SELECT firstName, lastName
           FROM song_history
           WHERE song = 'All Hands Against His Own'
           """

# execute
try:
    rows = session.execute(query3)
except Exception as e:
    print(e)
    
# print
for row in rows:
    print(row)
    print("*" * 100)
                    

Row(firstname='Jacqueline', lastname='Lynch')
****************************************************************************************************
Row(firstname='Tegan', lastname='Levine')
****************************************************************************************************
Row(firstname='Sara', lastname='Johnson')
****************************************************************************************************


## Drop the table before closing out the sessions

In [26]:
app_history_drop_query = "DROP TABLE IF EXISTS app_history"
user_session_history_drop_query = "DROP TABLE IF EXISTS user_session_history"
song_history_drop_query = "DROP TABLE IF EXISTS song_history"

# execute
try:
    session.execute(app_history_drop_query)
    session.execute(user_session_history_drop_query)
    session.execute(song_history_drop_query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [27]:
session.shutdown()
cluster.shutdown()

## References

* https://docs.datastax.com/en/cql-oss/3.3/cql/cql_using/useQueryColumnsSort.html